#### Generators explained

Example take from https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

In [1]:
def simple_generator_function():
    yield 1
    yield 2
    yield 3

In [4]:
res = simple_generator_function()
type(res)

generator

In [6]:
next(res)

1

In [7]:
next(res)

2

In [8]:
next(res)

3

In [9]:
#Should give error
next(res)

StopIteration: 

In [10]:
#recreate
res = simple_generator_function()

In [11]:
[x for x in res]

[1, 2, 3]

In [12]:
#Should give nothing
[x for x in res]

[]

A complex example 

In [17]:
import math

def get_primes(input_list):
    result_list = list()
    for element in input_list:
        if is_prime(element):
            result_list.append()

    return result_list

# or better yet...

def get_primes(input_list):
    return (element for element in input_list if is_prime(element))

# not germane to the example, but here's a possible implementation of
# is_prime...

def is_prime(number):
    if number > 1:
        if number == 2:
            return True
        if number % 2 == 0:
            return False
        for current in range(3, int(math.sqrt(number) + 1), 2):
            if number % current == 0: 
                return False
        return True
    return False

*** get_primes ***  works, but suppose we want infinite number of primes to be generated, and get_primes to be called for such infinte list, it does not seem possible in this framework. We need to discover a mechanism where we can simple generate primes one at a time and and keep on calling the *** is_prime *** function. By doing this we do not need to create a function generating primes infinitely..

In [18]:
def get_primes(number):
    while True:
        if is_prime(number):
            yield number
        number += 1

In [19]:
def solve_number_10():
    # She *is* working on Project Euler #10, I knew it!
    total = 2
    for next_prime in get_primes(3):
        if next_prime < 2000000:
            total += next_prime
        else:
            print(total)
            return

In [20]:
solve_number_10()

142913828922


Let me try to combine return along with yield

In [28]:
def get_primes_to_limit(number, limit):
    while True:
        if is_prime(number):
            yield number
        number += 1
        if(number > limit):
            print("Limit limit={} number={}".format(limit, number) + " reached. Returning")
            return(-100)
    return(-1)

In [35]:
def solve_number_10():
    # She *is* working on Project Euler #10, I knew it!
    total = 2
    for next_prime in get_primes_to_limit(3, 20):
        print("solve_number_10: next_prime={}".format(next_prime))
        if next_prime < 2000000:
            total += next_prime
        else:
            print(total)
    return(total)

In [36]:
solve_number_10()

solve_number_10: next_prime=3
solve_number_10: next_prime=5
solve_number_10: next_prime=7
solve_number_10: next_prime=11
solve_number_10: next_prime=13
solve_number_10: next_prime=17
solve_number_10: next_prime=19
Limit limit=20 number=21 reached. Returning


77

https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

In PEP 342, support was added for passing values into generators. PEP 342 gave generators the power to yield a value (as before), receive a value, or both yield a value and receive a (possibly different) value in a single statement.

In [37]:
def get_primes(number):
    while True:
        if is_prime(number):
            number = yield number
        number += 1
        
        
def print_successive_primes(iterations, base=10):
    prime_generator = get_primes(base)
    prime_generator.send(None)
    for power in range(iterations):
        print(prime_generator.send(base ** power))

In [41]:
print_successive_primes(4, base=10)

2
11
101
1009


In [ ]:
import random

def get_data():
    """Return 3 random integers between 0 and 9"""
    return random.sample(range(10), 3)

def consume():
    """Displays a running average across lists of integers sent to it"""
    running_sum = 0
    data_items_seen = 0

    while True:
        data = yield
        data_items_seen += len(data)
        running_sum += sum(data)
        print('The running average is {}'.format(running_sum / float(data_items_seen)))

def produce(consumer):
    """Produces a set of values and forwards them to the pre-defined consumer
    function"""
    while True:
        data = get_data()
        print('Produced {}'.format(data))
        consumer.send(data)
        yield

if __name__ == '__main__':
    consumer = consume()
    consumer.send(None)
    producer = produce(consumer)

    for _ in range(10):
        print('Producing...')
        next(producer)